# 7b) Clustering

In [1]:
import sklearn
import plotly.plotly as py
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from plotly import figure_factory as figureFact
from sklearn.cluster import KMeans 

#use pandas to load csv file
import pandas as pandas
#use numpy to calculate stuff
import numpy as np

# init offline plotlib
init_notebook_mode(connected=True)

In [2]:
#load data
redwinedata = pandas.read_csv('data/red_small.csv', sep =';')
whitewinedata = pandas.read_csv('data/white_small.csv', sep =';')
concat_data = redwinedata.append(whitewinedata)

In [3]:
# drop the quality label and data
concat_data = concat_data.drop('quality', axis=1)

In [4]:
#merge both tables to one
concat_table = figureFact.create_table(concat_data)
py.iplot(concat_table, filename='winequality')

In [5]:
concat_array = concat_table.values
sklearn.preprocessing.scale(concat_array)


TypeError: float() argument must be a string or a number, not 'builtin_function_or_method'

preprocessing.scale in sklearn to normalize

In [ ]:
kmeans = KMeans(n_clusters=2) 
kmeans.fit(concat_table)